In [1]:
!pip install pyspark #jalankan kode jika menggunakan google colab


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=707fd758d30370b07160a0c1fc48b63a0db9f6502999609ece374c069c4f3e44
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan6').getOrCreate()

data = [('James', 'Sales', 3000),
        ('Michael', 'Sales', 4600),
        ('Robert', 'Sales', 4100),
        ('Maria', 'Finance', 3000)]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, schema=columns)
df.show()

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|       James|     Sales|  3000|
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
|       Maria|   Finance|  3000|
+------------+----------+------+



In [ ]:
# Contoh operasi transformasi DataFrame
df.select('EmployeeName', 'Salary').show()
df.filter(df['Salary'] > 3000).show()
df.groupBy('Department').avg('Salary').show()


+------------+------+
|EmployeeName|Salary|
+------------+------+
|       James|  3000|
|     Michael|  4600|
|      Robert|  4100|
|       Maria|  3000|
+------------+------+

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
+------------+----------+------+

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|     Sales|     3900.0|
|   Finance|     3000.0|
+----------+-----------+



In [ ]:
# Contoh manipulasi tipe data kompleks
df1 = df.withColumn('SalaryBonus', df['Salary'] * 0.1)
df1.show()
df1.withColumn('TotalCompensation', df1['Salary'] + df1['SalaryBonus']).show()

+------------+----------+------+-----------+
|EmployeeName|Department|Salary|SalaryBonus|
+------------+----------+------+-----------+
|       James|     Sales|  3000|      300.0|
|     Michael|     Sales|  4600|      460.0|
|      Robert|     Sales|  4100|      410.0|
|       Maria|   Finance|  3000|      300.0|
+------------+----------+------+-----------+

+------------+----------+------+-----------+-----------------+
|EmployeeName|Department|Salary|SalaryBonus|TotalCompensation|
+------------+----------+------+-----------+-----------------+
|       James|     Sales|  3000|      300.0|           3300.0|
|     Michael|     Sales|  4600|      460.0|           5060.0|
|      Robert|     Sales|  4100|      410.0|           4510.0|
|       Maria|   Finance|  3000|      300.0|           3300.0|
+------------+----------+------+-----------+-----------------+



In [ ]:
# Contoh menggunakan window functions
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Department').orderBy('Salary')
df.withColumn('Rank', F.rank().over(windowSpec)).show()

windowval = (Window.partitionBy('Department').orderBy('Salary').rangeBetween(Window.unboundedPreceding, 0))
df_w_cumsum = df.withColumn('cum_sum', F.sum('Salary').over(windowval))
df_w_cumsum.show()

# Define window specification
windowSpec = Window.orderBy("EmployeeName").rowsBetween(Window.unboundedPreceding, Window.currentRow)
# Add running total column
df = df.withColumn("Running_Total", F.sum("Salary").over(windowSpec))
# Show the result
df.show()

+------------+----------+------+----+
|EmployeeName|Department|Salary|Rank|
+------------+----------+------+----+
|       Maria|   Finance|  3000|   1|
|       James|     Sales|  3000|   1|
|      Robert|     Sales|  4100|   2|
|     Michael|     Sales|  4600|   3|
+------------+----------+------+----+

+------------+----------+------+-------+
|EmployeeName|Department|Salary|cum_sum|
+------------+----------+------+-------+
|       Maria|   Finance|  3000|   3000|
|       James|     Sales|  3000|   3000|
|      Robert|     Sales|  4100|   7100|
|     Michael|     Sales|  4600|  11700|
+------------+----------+------+-------+

+------------+----------+------+-------------+
|EmployeeName|Department|Salary|Running_Total|
+------------+----------+------+-------------+
|       James|     Sales|  3000|         3000|
|       Maria|   Finance|  3000|         6000|
|     Michael|     Sales|  4600|        10600|
|      Robert|     Sales|  4100|        14700|
+------------+----------+------+-----